<a href="https://colab.research.google.com/github/kidubo/01-weather-app/blob/master/PDF_Query_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.0 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "sk-key"

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# location of the pdf file/files.
reader = PdfReader('/content/gdrive/My Drive/SURE CREDIT/Unlocking-Mechanisms-for-Greater-Youth-Financial-Inclusion.pdf')

In [ ]:
reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
raw_text

'PAGE 1\nPOWERED BY\nUnlocking mechanisms \nfor greater youth  \nfinancial INCLUSIONFSDT FOCUS NOTE\nNeema Mosha, Advisor - Communications, FSDT      \nVictor Kyando, Officer - Communications, FSDTFebruary 2021Standing at 1.2 billion, young people account for 1 in 6 of \nthe global population1 and are growing rapidly in number. \nHowever, despite the size of this age group, youth face \nsignificant economic and social barriers to inclusion within \neducation, employment, health services, civic participation \nand the financial sector. \nFinancial exclusion limits young people from benefitting from \na wide range of products and services that can help them build \nhousehold income and resilience, protect, and strengthen \ntheir families, weather unexpected financial shocks and \nimprove their livelihoods. Unlocking mechanisms for  \ngreater youth financial inclusionINTRODUCTION\nAGE DISTRIBUTION\n16-24 25-34 35-44 45-54 55-64 65-older26% 26%\n19%\n14%\n7%8%\nSource: FinScope Tanzania 20

In [ ]:
raw_text[:100]

'PAGE 1\nPOWERED BY\nUnlocking mechanisms \nfor greater youth  \nfinancial INCLUSIONFSDT FOCUS NOTE\nNeema'

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

10

In [ ]:
texts[0]

'PAGE 1\nPOWERED BY\nUnlocking mechanisms \nfor greater youth  \nfinancial INCLUSIONFSDT FOCUS NOTE\nNeema Mosha, Advisor - Communications, FSDT      \nVictor Kyando, Officer - Communications, FSDTFebruary 2021Standing at 1.2 billion, young people account for 1 in 6 of \nthe global population1 and are growing rapidly in number. \nHowever, despite the size of this age group, youth face \nsignificant economic and social barriers to inclusion within \neducation, employment, health services, civic participation \nand the financial sector. \nFinancial exclusion limits young people from benefitting from \na wide range of products and services that can help them build \nhousehold income and resilience, protect, and strengthen \ntheir families, weather unexpected financial shocks and \nimprove their livelihoods. Unlocking mechanisms for  \ngreater youth financial inclusionINTRODUCTION\nAGE DISTRIBUTION\n16-24 25-34 35-44 45-54 55-64 65-older26% 26%\n19%\n14%\n7%8%\nSource: FinScope Tanzania 20

In [ ]:
texts[1]

'Source: Tanzania population demographics www.pristech.ltdUnderstanding Millenials & Gen-Z•They want to live now •They are digital citizens •Prefer short term contracts than lifetime contracts •They are driven by trends than culture •They don’t have ﬁxed assets •They don’t have credit history •They do small but many transactions\nwww.pristech.ltdSuccessful use casesIn Tanzania services like Songesha have validated the need for micro loans in form of buy now pay later model where as per Vodacom annual report 2022, Songesha has offered about Tsh562 billion with annual growth of 22.4% in revenues from last year and 40% growth in number of beneﬁciaries from last year. Source: Vodacom Tanzania annual report'

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the document?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The authors of the document are Financial Inclusion of Youth, United Nations Youth, 2012, Arif Noorally Fazel, and Tanzania Youth Vision Association, Political, Educational and Communications Consultant.'

In [ ]:
query = "What was the presentation is about summurise in 5 sentenses"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The presentation was about raising awareness of the challenges youth face when trying to access financial services. It focused on engaging with 16 to 24-year-olds, including those living in rural areas. Storytelling, radio and TV dramas, university debates, and mobile video units (MVUs) were used to engage with the target audience. The presentation also highlighted the success of the IBUA! debate in increasing visibility and creating partnerships. Finally, the presentation discussed how youth can become empowered to act as agents of change and break the cycle of poverty.'

In [ ]:
query = "what are the Challenges to developing youth market"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The challenges to developing the youth market include restrictions in the legal and regulatory environment, inappropriate and inaccessible products and services, and low financial capabilities.'

In [ ]:
query = "what the solutions to the Challenges to developing youth market"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The solutions to the challenges to developing the youth market include engaging government, financial service providers, youth serving organisations, and other youth stakeholders in a multi-stakeholder approach, as well as using media to promote key messages, raise awareness, and encourage dialogue.'

In [ ]:
query = "what need to be changed to make it general not for mwanga hakika only"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The proposal seeks to make changes to the credit scoring methodology to make it applicable to other financial institutions in the market, not just Mwanga Hakika. The underlying technology and financial resources would need to be adjusted to accommodate this change.'

In [ ]:
query = "what would make this document even more convicing to mwanga hakika"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" This document could be made more convincing to Mwanga Hakika by providing more detailed examples of how the Sure Credit system will positively impact the bank's ability to offer micro-loans, BNPL products, credit cards, and unsecured personal loans. Additionally, providing data that supports the claim of a potential 2 trillion shillings in lost opportunity due to difficulties in assessing creditworthiness in Tanzania could further convince Mwanga Hakika of the value of the Sure Credit system."